# The `DataFrame`

The DataFrame data structure is the heart of the Panda's library. It's a primary object that you'll be working  with in data analysis and cleaning tasks.

The DataFrame is conceptually a two-dimensional series object, where there's an index and multiple columns of  content, with each column having a label. In fact, the distinction between a column and a row is really only a  conceptual distinction. And you can think of the DataFrame itself as simply a two-axes labeled array.

In [1]:
# Lets start by importing our pandas library
import pandas as pd

We create three school records for students and their 
class grades. Each record is a series which has a student name, the class name, and the score.

In [2]:
record1 = pd.Series({'Name': 'Alice',
                        'Class': 'Programming',
                        'Score': 85})
record2 = pd.Series({'Name': 'Bob',
                        'Class': 'Cryptography',
                        'Score': 82})
record3 = pd.Series({'Name': 'Carol',
                        'Class': 'Networking',
                        'Score': 90})

Like a Series, the DataFrame object is index. Here we use a group of series, where each series 
represents a row of data. Just like the Series function, we can pass in our individual items
in an array, and we can pass in our index values as a second arguments

In [3]:
df = pd.DataFrame([record1, record2, record3],
                  index=['stream1', 'stream2', 'stream1'])

# And just like the Series we can use the head() function to see the first several rows of the
# dataframe, including indices from both axes, and we can use this to verify the columns and the rows
df.head()

,Name,Class,Score
stream1,Alice,Programming,85
stream2,Bob,Cryptography,82
stream1,Carol,Networking,90


Note that Jupyter formats the output with HTML to render the results of the
dataframe. So we have the index, which is the leftmost column and is the school name, and then we have the rows of data, where each row has a column header which was given in our initial record dictionaries.

An alternative method is that you could use a list of dictionaries, where each dictionary 
represents a row of data.

In [4]:
students = [{'Name': 'Alice',
              'Class': 'Programming',
              'Score': 85},
            {'Name': 'Bob',
             'Class': 'Cryptography',
             'Score': 82},
            {'Name': 'Carol',
             'Class': 'Networking',
             'Score': 90}]

# Then we pass this list of dictionaries into the DataFrame function
df = pd.DataFrame(students, index=['stream1', 'stream2', 'stream1']) 
# And lets print the head again
df.head()

,Name,Class,Score
stream1,Alice,Programming,85
stream2,Bob,Cryptography,82
stream1,Carol,Networking,90


Similar to the series, we can extract data using the `.iloc` and `.loc` attributes. Because the DataFrame is two-dimensional, passing a single value to the loc indexing operator will return the series if there's only one row to return.

For instance, if we wanted to select data associated with stream2, we would just query the `.loc` attribute with one parameter.

In [5]:
df.loc['stream2']

Name              Bob
Class    Cryptography
Score              82
Name: stream2, dtype: object

You'll note that the name of the series is returned as the index value, while the column 
name is included in the output.

We can check the data type of the return using the python type function.

In [6]:
type(df.loc['stream2'])

pandas.core.series.Series

It's important to remember that the index values and column names along either axes, horizontal or vertical, could be non-unique. In this example, we see two records for school1 as different rows.

If we use a single value with the DataFrame `.loc` attribute, multiple rows of the DataFrame will return, not as a new series, but as a new DataFrame.

Lets query for stream1 records

In [7]:
df.loc['stream1']

,Name,Class,Score
stream1,Alice,Programming,85
stream1,Carol,Networking,90


In [8]:
# And we can see the the type of this is different too
type(df.loc['stream1'])

pandas.core.frame.DataFrame

It is easy to select data from a dataframe based on multiple axes. For instance, if we wanted to just list the student names for stream1, we would supply two arguments to `.loc`, one being the row index and the second being the column name.

For instance, if we are only interested in stream1's student names

In [9]:
df.loc['stream1', 'Name']

stream1    Alice
stream1    Carol
Name: Name, dtype: object

Remember, just like the Series, the pandas developers have implemented this using the indexing operator `.loc[..]` and not as parameters to a function.

What would we do if we just wanted to select a single column though? 

The first way is to transpose the dataframe and apply row operations. This pivots all of the rows into columns and all of the columns into rows, and is done with the `.T` attribute

In [10]:
df.T

,stream1,stream2,stream1
Name,Alice,Bob,Carol
Class,Programming,Cryptography,Networking
Score,85,82,90


In [11]:
# Then we can call .loc on the transpose to get the student names only
df.T.loc['Name']

stream1    Alice
stream2      Bob
stream1    Carol
Name: Name, dtype: object

However, since `iloc` and `loc` are used for row selection, Panda reserves the indexing operator directly on the DataFrame for column selection. In a Panda's DataFrame, columns always have a name.

So this selection is always label based, and is not as confusing as it was when using the square bracket operator on the series objects. For those familiar with relational databases, this operator is analogous to column projection.

In [12]:
df['Name']

stream1    Alice
stream2      Bob
stream1    Carol
Name: Name, dtype: object

In practice, this works really well since you're often trying to add or drop new columns. However,
this also means that you get a key error if you try and use .loc with a column name

In [13]:
df.loc['Name']

KeyError: 'Name'

Note too that the result of a single column projection is a Series object

In [14]:
type(df['Name'])

pandas.core.series.Series

Since the result of using the indexing operator is either a DataFrame or Series, you can chain operations together. For instance, we can select all of the rows which related to stream1 using `.loc`, then project the name column from just those rows.

In [15]:
df.loc['stream1']['Name']

stream1    Alice
stream1    Carol
Name: Name, dtype: object

If you get confused, use `type()` to check the responses from resulting operations


In [16]:
print(type(df.loc['stream1'])) #should be a DataFrame
print(type(df.loc['stream1']['Name'])) #should be a Series

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


Chaining, by indexing on the return type of another index, can come with some costs and is best avoided if you can use another approach. In particular, chaining tends to cause Pandas to return a copy of the DataFrame instead of a view on the DataFrame. For selecting data, this is not a big deal, though it might be slower than necessary. If you are changing data though this is an important distinction and can be a source of error.

Here's another approach. As we saw, .loc does row selection, and it can take two parameters, the row index and the list of column names. The .loc attribute also supports slicing.

If we wanted to select all rows, we can use a colon to indicate a full slice from beginning to end. This is just like slicing characters in a list in python. Then we can add the column name as the second parameter as a string. If we wanted to include multiple columns, we could do so in a list. 
and Pandas will bring back only the columns we have asked for.

Here's an example, where we ask for all the names and scores for all streams using the .loc operator.

In [17]:
df.loc[:,['Name', 'Score']]

,Name,Score
stream1,Alice,85
stream2,Bob,82
stream1,Carol,90


Note that the colon means that we want to get all of the rows, and the list
in the second argument position is the list of columns we want to get back

That's selecting and projecting data from a DataFrame based on row and column labels. The key concepts to remember are that the rows and columns are really just for our benefit. Underneath this is just a two axes labeled array, and transposing the columns is easy. Also, consider the issue of chaining carefully, and try to avoid it, as it can cause unpredictable results, where your intent was to obtain a view of the data, but instead Pandas returns to you a copy. 

Before we leave the discussion of accessing data in DataFrames, lets talk about dropping data. It's easy to delete data in Series and DataFrames, and we can use the drop function to do so. This function takes a single parameter, which is the index or row label, to drop. This is another 
tricky place for new users -- the drop function doesn't change the DataFrame by default! Instead, the drop function returns to you a copy of the DataFrame with the given rows removed.

In [18]:
df.drop('stream1')

,Name,Class,Score
stream2,Bob,Cryptography,82


But if we look at our original DataFrame we see the data is still intact.

In [19]:
df

,Name,Class,Score
stream1,Alice,Programming,85
stream2,Bob,Cryptography,82
stream1,Carol,Networking,90


Drop has two interesting optional parameters. The first is called inplace, and if it's 
set to true, the DataFrame will be updated in place, instead of a copy being returned. 
The second parameter is the axes, which should be dropped. By default, this value is 0, 
indicating the row axis. But you could change it to 1 if you want to drop a column.

For example, lets make a copy of a DataFrame using .copy()

In [20]:
copy_df = df.copy()
# Now lets drop the name column in this copy
copy_df.drop("Name", inplace=True, axis=1)
copy_df

,Class,Score
stream1,Programming,85
stream2,Cryptography,82
stream1,Networking,90


There is a second way to drop a column, and that's directly through the use of the indexing operator, using the `del` keyword. This way of dropping data, however, takes immediate effect on the DataFrame and does not return a view.

In [21]:
del copy_df['Class']
copy_df

,Score
stream1,85
stream2,82
stream1,90


Finally, adding a new column to the DataFrame is as easy as assigning it to some value using
the indexing operator. For instance, if we wanted to add a class ranking column with default 
value of None, we could do so by using the assignment operator after the square brackets.
This broadcasts the default value to the new column immediately.

In [22]:
df['ClassRanking'] = None
df

,Name,Class,Score,ClassRanking
stream1,Alice,Programming,85,None
stream2,Bob,Cryptography,82,None
stream1,Carol,Networking,90,None


# Summary

We have learned about the data structure we will use the most in pandas, the DataFrame. The  dataframe is indexed both by row and column, and we can easily select individual rows and project the columns we are interested in using the familiar indexing methods from the Series class.